In [1]:
import torch
from common.cache import CacheType
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

TEST_CACHE = CacheType.READ

### Test in UBFC-rPPG

In [2]:
from common.cache import CacheType
from data_generator.PhysNet import PhysNetDataConfig, PhysNetDataGenerator
from dataset_reader.UBFC_rPPG import UBFCrPPGsDatasetReader
from PhysNet_Train_in_UBFC_rPPG import \
    T,WIDTH,HEIGHT,BATCH,sbs_physnet as sbs

TEST_NAME = "Test_in_UBFC_rPPG"
MODEL_NAME = "PhysNet_Train_in_UBFC_rPPG"
test_dataset_path = r"/public/share/weiyuanwang/dataset/UBFC-rPPG"
test_cache_path = r"~/cache/PhysNet/UBFC-rPPG/test"
test_dataset_reader = UBFCrPPGsDatasetReader(test_dataset_path,dataset=2,dataset_list=[
    'subject49', 'subject5'
])
test_dataset_config = PhysNetDataConfig(
    cache_root=test_cache_path,
    cache_type=TEST_CACHE,batch_size=BATCH,
    step=T,width=WIDTH,height=HEIGHT,slice_interval=T,
    num_workers=12,generate_num_workers=12,
    discard_front = 35,discard_back = 105,
)
test_dataset_generator = PhysNetDataGenerator(config=test_dataset_config)
test_raw_data = test_dataset_reader.read() if TEST_CACHE == CacheType.NEW_CACHE else None
test_dataloader = test_dataset_generator.generate(test_raw_data)


dataset size: 26


### Draw BVP

In [11]:
from common.cuda_info import get_device


save_path = f'out/img/test/[{MODEL_NAME}]_[{TEST_NAME}]/item'
map_save_path = f'out/img/test/[{MODEL_NAME}]_[{TEST_NAME}]/map'
os.makedirs(save_path, exist_ok=True)
os.makedirs(map_save_path, exist_ok=True)
def draw_pred_true(y_1,y_2,name):
    plt.figure(figsize=(8,6))
    plt.xlabel('Frame')
    plt.ylabel('BVP Strength')
    ax = plt.gca()
    for spine in ax.spines:
        ax.spines[spine].set_visible(False)
    plt.plot(y_1,label='True BVP')
    plt.plot(y_2,label='Predicted BVP')
    plt.legend(loc='upper right')
    plt.savefig(f'{save_path}/{name}.pdf', transparent=True, bbox_inches='tight')
    plt.savefig(f'{save_path}/{name}.svg', transparent=True, bbox_inches='tight')
    plt.close('all')
draw_y = []
draw_y_p = []
min_loss = 10
with torch.no_grad():
    model = sbs.model
    loss_fn = sbs.loss_fn
    device = get_device()
    for i, (X, y) in enumerate(test_dataloader):
        x_tensor = torch.as_tensor(X).float().to(device)
        y_tensor = torch.as_tensor(y).float().to(device)
        for b_index in range(X.shape[0]):
            b_x = x_tensor[b_index].unsqueeze(0)
            b_y = y_tensor[b_index].unsqueeze(0)
            y_pred = model(b_x)
            x_visual6464 = model.x_visual6464(b_x)

            loss = loss_fn(y_pred,b_y).item()
            y_pred = y_pred.squeeze(0).detach().cpu().numpy()
            b_y = b_y.squeeze(0).detach().cpu().numpy()
            frame_raw = b_x.squeeze(0).permute((1,2,3,0))[0]
            frame_raw = (frame_raw*255).int().cpu().numpy()
            draw_pred_true(b_y,y_pred,f'{i}_{b_index}')
            frame = x_visual6464.squeeze(0).permute((1,2,3,0))[0]
            frame = frame.mean(axis=2)
            frame  = (frame - frame.min())/(frame.max()-frame.min()) * 255
            frame = frame.int().cpu().numpy()
            cv2.imwrite(f'{map_save_path}/{i}_{b_index}.jpg',frame)
            cv2.imwrite(f'{map_save_path}/raw_{i}_{b_index}.jpg',frame_raw)


            if min_loss > loss:
                min_loss = loss
                draw_y_p = y_pred
                draw_y = b_y
draw_pred_true(draw_y,draw_y_p,'best')